In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 13.4 MB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
     |████████████████████████████████| 3.3 MB 45.6 MB/s 
     |████████████████████████████████| 596 kB 43.3 MB/s 
     |████████████████████████████████| 59 kB 7.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install gensim==4.1.2

     |████████████████████████████████| 24.1 MB 9.8 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
titles_questions = []
labels = []
data_path = "/content/drive/MyDrive/Zalo 2021/Training Data/training_draft_bm25_vers_short.json"
with open(data_path) as json_file:
    data = json.load(json_file)
for question in data:
    titles_questions.append([question['question'], question['article']['title'] + question['article']['text']])
    labels.append(question['label'])
print(len(labels), len(titles_questions))

9988 9988


In [ ]:
from sklearn.model_selection import train_test_split
train_titles_questions, val_titles_questions, train_labels, val_labels = train_test_split(titles_questions, labels, test_size=.2)

In [ ]:
print(train_titles_questions[0])

['Mức phạt khi người được chở trên xe mô tô, xe gắn máy đội mũ bảo hiểm không cài quai đúng cách', 'Điều 29. Xử phạt các hành vi vi phạm quy định về tham gia bảo hiểm của người điều khiển xe cơ giới1. Phạt tiền 100.000 đồng đối với người điều khiển xe mô tô, xe gắn máy, xe mô tô ba bánh và các loại xe cơ giới tương tự không có hoặc không mang theo Giấy chứng nhận bảo hiểm còn hiệu lực.\n2. Phạt tiền 500.000 đồng đối với người điều khiển xe ô tô, máy kéo và các loại xe cơ giới tương tự không có hoặc không mang theo Giấy chứng nhận bảo hiểm còn hiệu lực.']


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('nguyenvulebinh/vi-mrc-base')

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(train_titles_questions, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_titles_questions, truncation=True, padding=True,max_length=512)

In [ ]:
print(tokenizer.convert_ids_to_tokens(train_encodings['input_ids'][100]))

['<s>', '▁Khi', 'ếu', '▁nạ', 'i', '▁bằng', '▁cách', '▁gửi', '▁email', '▁có', '▁được', '▁không', '?', '</s>', '</s>', '▁Điều', '▁6.', '▁Hình', '▁thức', '▁khiếu', '▁nạ', 'i', '1', '.', '▁Khi', 'ếu', '▁nạ', 'i', '▁thực', '▁hiện', '▁bằng', '▁hình', '▁thức', '▁gửi', '▁đơn', '▁khiếu', '▁nạ', 'i', '▁hoặc', '▁khiếu', '▁nạ', 'i', '▁trực', '▁tiếp', ',', '▁được', '▁quy', '▁định', '▁như', '▁sau', ':', '▁a', ')', '▁Khi', 'ếu', '▁nạ', 'i', '▁bằng', '▁hình', '▁thức', '▁gửi', '▁đơn', '▁thì', '▁trong', '▁đơn', '▁khiếu', '▁nạ', 'i', '▁ghi', '▁rõ', '▁nội', '▁dung', '▁sau', '▁đây', ':', '▁ngày', ',', '▁tháng', ',', '▁năm', '▁khiếu', '▁nạ', 'i', ';', '▁tên', ',', '▁địa', '▁chỉ', '▁của', '▁người', '▁khiếu', '▁nạ', 'i', ';', '▁tên', ',', '▁địa', '▁chỉ', '▁của', '▁cơ', '▁quan', ',', '▁tổ', '▁chức', ',', '▁cá', '▁nhân', '▁bị', '▁khiếu', '▁nạ', 'i', ';', '▁nội', '▁dung', ',', '▁lý', '▁do', '▁khiếu', '▁nạ', 'i', ',', '▁tài', '▁liệu', '▁liên', '▁quan', '▁đến', '▁nội', '▁dung', '▁khiếu', '▁nạ', 'i', '▁(', 'n', 'ếu

In [ ]:
print(len(val_encodings[0]))

512


In [ ]:
import torch

class Zalo2021(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Zalo2021(train_encodings, train_labels)
val_dataset = Zalo2021(val_encodings, val_labels)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1} 

In [ ]:
# from torch.utils.data import DataLoader
# from transformers import AdamW
# from transformers import AutoModelForSequenceClassification

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# model = AutoModelForSequenceClassification.from_pretrained("nguyenvulebinh/vi-mrc-base")
# model.to(device)
# optim = AdamW(model.parameters(), lr=5e-5)


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    do_train=True,
    do_eval=True,
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.005,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_steps = 200,
    evaluation_strategy="steps",


)

model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Zalo 2021/Pretrained models/checkpoint-5000')


In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics = compute_metrics
)

trainer.train()

***** Running training *****
  Num examples = 7990
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2997


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
200,0.240400,0.339028,0.900901,0.788618,0.570588,0.662116
400,0.555800,0.326592,0.883383,0.805714,0.414706,0.547573
600,0.340500,0.318373,0.905906,0.765734,0.644118,0.699681
800,0.201600,0.290934,0.900901,0.743151,0.638235,0.686709
1000,0.302900,0.319052,0.883383,0.639687,0.720588,0.677732
1200,0.122200,0.362588,0.906406,0.754153,0.667647,0.708268
1400,0.114400,0.321629,0.909910,0.791971,0.638235,0.706840
1600,0.217500,0.320662,0.902402,0.712610,0.714706,0.713656
1800,0.369100,0.281733,0.908909,0.740854,0.714706,0.727545
2000,0.264200,0.293302,0.915415,0.758308,0.738235,0.748137


***** Running Evaluation *****
  Num examples = 1998
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1998
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1998
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1998
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1998
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1998
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1998
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/chec

KeyboardInterrupt: ignored

In [ ]:
# Save model    
tokenizer.save_pretrained("/content/drive/MyDrive/Zalo 2021/models")
model.save_pretrained("/content/drive/MyDrive/Zalo 2021/models")

In [ ]:
import json
import torch
import numpy as np
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
data_path = "/content/drive/MyDrive/Zalo 2021/Training Data/testing_bm25_top10.json"
model.to(device)
with open(data_path) as json_file:
    data = json.load(json_file)
to_save = []
model.eval()
for question in data:
    relevant_articles = []
    for article in question['relevant_articles']:
        # if len(article['text'].split()) < 450:
        encodings = tokenizer([[question['question'], article['title'] + article['text']]], truncation=True, padding=True, max_length=512, return_tensors= 'pt')
        input_ids = encodings['input_ids'].to(device)
        attention_mask = encodings['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs['logits']
        logits = logits.detach().cpu().numpy()
        pred = np.argmax(logits, axis=1)
        if pred == 1:
            article_pred = {}
            article_pred['law_id'] = article['law_id']
            article_pred['article_id'] = article['article_id']
            relevant_articles.append(article_pred)
        # else:
        #     para_list = article['text'].split('\n')
        #     for para in para_list:
        #         encodings = tokenizer([[question['question'], article['title'] + para]], truncation=True, padding=True, max_length=512, return_tensors= 'pt')
        #         input_ids = encodings['input_ids'].to(device)
        #         attention_mask = encodings['attention_mask'].to(device)
        #         outputs = model(input_ids, attention_mask=attention_mask)
        #         logits = outputs['logits']
        #         logits = logits.detach().cpu().numpy()
        #         pred = np.argmax(logits, axis=1)
        #         if pred == 1:
        #             article_pred = {}
        #             article_pred['law_id'] = article['law_id']
        #             article_pred['article_id'] = article['article_id']
        #             relevant_articles.append(article_pred)
        #             break
    question_pred = {}
    question_pred['question_id'] = question['id']
    question_pred['relevant_articles'] = relevant_articles
    to_save.append(question_pred)

In [ ]:
import codecs
import os
dest_path = '/content/drive/MyDrive/Zalo 2021'
today = 'submission06.json'

file_path = os.path.join(dest_path, today)
with codecs.open(file_path, "w", encoding='utf8') as outfile:
        json.dump(to_save   , outfile, ensure_ascii=False)